# Integration via the Holmes-Diaconis-Ross algorithm using `LinConGauss`

### Outline
This notebook shows how to use the `LinConGauss` package to estimate the integral of a linearly constrained Gaussian.
The procedure is the following:
- define the linear constraints
- run subset simulation to determine the nestings
- run Holmes-Diaconis-Ross to get an unbiased estimate of the integral

Details on the method can be found in [Gessner, Kanjilal, and Hennig: Integrals over Gaussians under Linear Domain Constraints](https://arxiv.org/abs/1910.09328)

__This example__  
Consider a 100d shifted orthant in a standard normal space. The integral is available in closed form
We compute the integral using `LinConGauss` and compare to the ground truth.

_tutorial by Alexandra Gessner, Feb 2020_

In [1]:
import numpy as np
import LinConGauss as lcg

### Setting up linear constraints
The linear constraints are defined as the roots of $M$ (here `n_lc`) linear functions
$$\mathbf{f}(\mathbf{x}) = A_m^\intercal \mathbf{x} + \mathbf{b}. $$
and the domain of interest is defined as the intersection of where all these functions are _positive_.

In this setting we assume the linear constraints to be axis-aligned.

In [2]:
# Problem dimension
dim = 100

# seed for reproducibility
np.random.seed(0)

# number of linear constraints
n_lc = np.copy(dim)

# generate random linear constraints
A = np.eye(n_lc)
b = np.random.randn(n_lc, 1)

# define the linear constraints with LinConGauss
lincon = lcg.LinearConstraints(A=A, b=b)

### Ground truth integral
The ground truth integral is just the Gaussian CDF evaluated at $b$ since
$$\int_{-b}^\infty \mathcal{N} (x; 0,1) \mathrm{d}x = \int_{-\infty}^b \mathcal{N} (x; 0,1) \mathrm{d}x
= \Phi(b)$$

In [3]:
from scipy.stats import norm

true_integral = norm.cdf(lincon.b).prod()
print(true_integral)

4.4346483368318176e-42


### Integral via LinConGauss

Use subset simulation (Au&Beck 2001) to determine a sequence of shifts s.t. 1/2 of the samples fall inside the next domain. Subset simulation can also help estimate the integral, but it is biased.

Therefore we then hand the obtained shift sequence to the HDR method, with which we draw more samples per nesting in order to obtain unbiased estimates of the conditional probability of each nesting.

In [4]:
subsetsim = lcg.multilevel_splitting.SubsetSimulation(linear_constraints=lincon,
                                                      n_samples=16,
                                                      domain_fraction=0.5,
                                                      n_skip=3)
subsetsim.run(verbose=False)

In [5]:
shifts = subsetsim.tracker.shift_sequence

In [6]:
hdr = lcg.multilevel_splitting.HDR(linear_constraints=lincon,
                                   shift_sequence=shifts,
                                   n_samples=512,
                                   n_skip=9,
                                   X_init=subsetsim.tracker.x_inits())
hdr.run(verbose=False)

In [7]:
hdr_integral = hdr.tracker.integral()
print(hdr_integral)

1.1209678655929519e-42


In [8]:
rel_error = np.abs(true_integral - hdr_integral)/(true_integral + hdr_integral)
print(rel_error)

0.5964559736492593


This means that the integral estimate is about 1 order of magnitude off, which is not a lot given the small scale of the problem.

### Sampling from the domain
We already got a few samples from the integration procedure, which we can reuuse for sampling.
The method uses rejection-free elliptical slice sampling to sample from the domain. Given we already know samples in the domain of interest, we do not need to run subset simulation to find these, but we can directly define the sampler.

In [9]:
# samples known from integration
X_int = hdr.tracker.X

# Elliptical slice sampler
sampler = lcg.sampling.EllipticalSliceSampler(n_iterations=1000,
                                              linear_constraints=lincon,
                                              n_skip=9,
                                              x_init=X_int)
sampler.run()

In [10]:
# here are the samples
sampler.loop_state.X

array([[ 0.95768   ,  1.00569932,  1.1046428 , ..., -0.2665558 ,
        -0.23618649, -0.44097909],
       [-0.24078086, -0.29294991, -0.26082635, ...,  0.56955891,
         0.50018578,  0.47523858],
       [-0.06859234,  0.03219652,  0.30989099, ..., -0.57417054,
        -0.52996829, -0.46241832],
       ...,
       [ 0.81298016,  0.89595533,  0.75319723, ...,  0.07584069,
         0.10103985, -0.00448645],
       [ 0.85103263,  0.84523668,  0.97622414, ...,  0.44899093,
         0.46062743,  0.29172688],
       [ 1.24265713,  1.27339369,  1.48414384, ..., -0.37260173,
        -0.39422942, -0.34019564]])

### Alan Genz's method (integration only)
In this specific case, the integration method used by Alan Genz can be applied.
This requires the linear constraints to be rewritable as (potentially open) box constraints of a general Gaussian.
If this is the case and if furthermore samples are not required, this method is the method of choice for the given integration task.

The method can be found in `scipy.stats.mvn` as `mvnun` routine. This directly calls the `FORTRAN` implementation of `MVNDST` [written by Alan Genz](http://www.math.wsu.edu/faculty/genz/homepage)

In [11]:
from scipy.stats import mvn

lower = -b.squeeze()
upper = np.inf * np.ones_like(lower)

mean = np.zeros((n_lc,))
cov = np.eye(n_lc)

mvn.mvnun(lower, upper, mean, cov)

(4.434648336831776e-42, 0)